In [1]:
#!pip install pymed

/bin/bash: pip: command not found


In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/piotr/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/piotr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from pymed import PubMed
import pandas as pd
import sklearn
import pickle
import os

import ipywidgets as wg
from IPython.display import display

In [3]:
myName = wg.Text(value='disease')
N_abstract = wg.IntSlider(description='No. abstracts:', max=10000)
display(myName,N_abstract)

Text(value='disease')

IntSlider(value=0, description='No. abstracts:', max=10000)

In [4]:
print(myName.value + ' is ' + str(N_abstract.value) + ' years old')

liver cancer is 6774 years old


In [5]:

    text = myName.value
    max1 =int(N_abstract.value)  # ilosc zapytan
    #text = [text]

    #wporwadzenie zapytania do pubmed
    pubmed = PubMed(tool="MyTool", email="p.karabowicz@gmail.com")
    results1 = pubmed.query(text, max_results=max1)

    #przeksztalcenie wynikow zapytania na data frame
    lista_abstract_3=[]
    for i in results1:
        lista_abstract_3.append(i.abstract)
    import pandas as pd
    df_abstract = pd.DataFrame(lista_abstract_3, columns = ['abstracts'])
    df_abstract['abstracts_lower'] = df_abstract['abstracts'].str.lower()
    df_abstract_1 = df_abstract.dropna() #datafraame wynikow do analizy

    ###predykcja
    #from keras.models import load_model
    #BASE_DIR = os.path.dirname(os.path.abspath('finalized_model1.sav'))
    #BASE_DIR1 = '/home/piotr/drugfinal/static'
    #finalized_model= load_model(os.path.join(BASE_DIR, 'static/finalized_model1.sav'))
    
    #finalized_model= os.path.join(BASE_DIR, 'static/finalized_model1.sav')
    import pickle
    rnd = pickle.load(open('./finalized_model.sav', 'rb'))
    
    from sklearn.feature_extraction.text import CountVectorizer
    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100)
    result1=count_vect.fit_transform(df_abstract_1['abstracts_lower'])
    result2 = rnd.predict(result1)
    df_abstract_1['class'] = result2 # dataframe z wynikami
    len_df = len(result2)

#unsupervised learning
    from gensim.models import Word2Vec
    #from nltk.corpus import stopwords
    #przygotowanie tekstu do osadzania slow
    #stop = stopwords.words('english')
    stop = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]
    from nltk.tokenize import word_tokenize
    df_abstract_1['abstracts_stop'] = df_abstract_1['abstracts_lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df_abstract_1['tokenized'] = df_abstract_1.apply(lambda row: nltk.word_tokenize(row['abstracts_lower']), axis=1)
    model_ted1 = Word2Vec(sentences=df_abstract_1['tokenized'], size=200, window=10, min_count=1, workers=4, sg=0)

    #ekstrackja slow najbardziej podobnych do protein i target
    keys = ['protein', 'target']
    most_sim = model_ted1.wv.most_similar(positive = keys, topn=1000)
    #utworzenie tablicy ze slownika
    most_sim_key = []
    for w, n in most_sim:
        most_sim_key.append(w)
    #tagowanie tekstu i filtrowanie wedlug tagow
    post_tag_list = nltk.pos_tag(most_sim_key)

    listNN = []
    for w , k in post_tag_list:
        if k == "NN":
            listNN.append(w)

    listJJ = []
    for w , k in post_tag_list:
        if k == "JJ":
            listJJ.append(w)

    lista = [listNN, listJJ]

    flat_list = []
    for sublist in lista:
        for item in sublist:
            flat_list.append(item)
    #filtrowanie wzgledem bazy bialek
    #BASE_DIR2 = os.path.dirname(os.path.abspath('lista_bialek_bez1.txt'))
    file = open('./lista_bialek_bez1.txt', "r")
    wprowadzony_tekst = file.read()

    wt = wprowadzony_tekst.split(',')

    wt2 =[]
    for w in wt:
        w = w.replace("[", "")
        w = w.replace("]", "")
        w= w.replace("'", "")
        wt2.append(w)

    wt2 = [x.lower() for x in wt2 ]
    tablica_in =[] #lista wyekstrahowanych bialek
    for w in flat_list:
        if w in wt2:
            tablica_in.append(w)

    # predykcja dla zapytan z tablica_in
    import sklearn
    from sklearn.feature_extraction.text import CountVectorizer
    import time

    def query(list_target):
        pubmed = PubMed(tool="MyTool", email="p.karabowicz@gmail.com")
        lista=[]
        for w in list_target:
            time.sleep(1)
            lista.append(pubmed.query(w, max_results=20))
        return lista

    def lista_bastract_pred1(lista):
        lista_abstract_pred=[]
        for n in lista:
                lista_abstract_pred.append(n.abstract)
        return lista_abstract_pred

    def percent_true(ynew2):
        percent_true = round((list(ynew2).count(1))/len(list(ynew2))*100,1)
        return percent_true

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import make_classification

    import pickle
    #rnd = pickle.load(open('C:\\Users\\UMB\\Desktop\\drugforest\\drugforest\\static\\finalized_model_32.sav', 'rb'))

    a = query(tablica_in)
    d = []
    for w in a:
        b = lista_bastract_pred1(w) ## ziterować
        d.append(b)

    d2=[]
    for g in d:
        d1 = list(filter(None.__ne__, g))
        d2.append(d1)

    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100)
    lista_y=[]
    for w in d2:
        result1=count_vect.fit_transform(w)
        result2 = rnd.predict(result1)
        lista_y.append(result2)

    yy = []
    for y in lista_y:
        yy.append(percent_true(y))

    import pandas as pd
    data_tuples = list(zip(tablica_in,yy))
    df_list = pd.DataFrame(data_tuples, columns=['Protein','Score'])

    df_list = df_list.sort_values('Score', ascending = False)
    #df_list = df_list.reset_index()

    df_abstract_1 = df_abstract_1[['abstracts', 'class']]


/home/piotr/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/piotr/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/piotr/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.3 when using version 0.23.1. This m

In [7]:
df_list[['Protein', 'Score']]


,Protein,Score
17,protease,75.0
34,apoa-i,75.0
15,dnmt1,75.0
19,lkb1,73.7
31,siglec-15,70.6
2,p38,70.0
38,egf,70.0
1,cyclin,70.0
18,bfgf,70.0
12,act,68.4


In [8]:
myName.value

'liver cancer'

In [45]:
#results11 = pubmed.query('liver cancer', max_results=10000)
#lista_abstract_11=[]
#for i in results11:
    #lista_abstract_11.append(i.abstract)

KeyboardInterrupt: 